## Load environment and check for folder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import subprocess
import os
base_dir = "/content/gdrive/MyDrive/final-project-paralelizacao-tecnica-de-meios-tons"
get_ipython().run_line_magic('cd', base_dir)

# assert that we have the exercise folder
assert 'repositorio' in os.listdir(), "Exercise folder not found"

Cuda de Técnicas de de Meios-Tons

In [ ]:
import os
import subprocess

# Caminhos principais
base_dir = os.getcwd()
exercise_dir = os.path.join(base_dir, "repositorio")
build_dir = os.path.join(exercise_dir, "build")

# Função para executar comandos de shell
def run_command(command, work_dir):
    return subprocess.run(
        command, shell=True, cwd=work_dir, 
        text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )

# Preparar build: limpar e criar pasta build/
print("🔧 Limpando diretório de build...")
run_command("cmake -E remove_directory build", exercise_dir)
run_command("cmake -E make_directory build", exercise_dir)

# Executar CMake e verificar erro
print("⚙️ Executando CMake...")
cmake_result = run_command("cmake ..", build_dir)
print("CMAKE STDOUT:\n", cmake_result.stdout)
print("CMAKE STDERR:\n", cmake_result.stderr)
assert cmake_result.returncode == 0, "Erro na configuração com CMake"

# Compilar com make
print("🧱 Compilando com make...")
make_result = run_command("make -j", build_dir)
print("MAKE STDOUT:\n", make_result.stdout)
print("MAKE STDERR:\n", make_result.stderr)
assert make_result.returncode == 0, "Erro na compilação (make)"

# Lista de imagens de teste (ppm)
test_images = [
    # "img/ppm/city.ppm",
    # "img/ppm/baboon_colorida.ppm",
    "img/ppm/monalisa.ppm",
    # "img/ppm/seagull.ppm",
    # "img/ppm/butterfly.ppm"
]

output_dir = os.path.join(exercise_dir, 'out')
os.makedirs(output_dir, exist_ok=True)

# Executar os testes
for i, image in enumerate(test_images, start=1):
    print(f"\n===== 🧪 Teste {i} - Imagem: {image} =====")

    exec_path = os.path.join(build_dir, "cuda_dithering")
    img_path = os.path.join(exercise_dir, image)
    time_file = os.path.join(build_dir, f"time_{i}.txt")

    # Garantir permissão de execução
    subprocess.run(f"chmod +x {exec_path}", shell=True)

    # Montar e executar comando do programa
    command = f"{exec_path} {img_path} -g"
    print(f"▶️ Executando: {command}")

    run = subprocess.run(
        command, shell=True, cwd=exercise_dir,
        stdout=subprocess.PIPE, stderr=subprocess.PIPE,
        text=True
    )

    # Mostrar resultados
    print("\n=== 📤 Saída do programa (stdout) ===")
    print(run.stdout)

    print("\n=== 🐞 Logs do programa (stderr) ===")
    print(run.stderr)

    if os.path.exists(time_file):
        with open(time_file) as f:
            runtime = f.read().strip()
        print(f"\n⏱ Tempo de execução: {runtime} segundos")
    else:
        print(f"\n⚠️ Falha ao medir tempo para {image}")
